# 実行環境の前提条件

本ディレクトリ配下にあるノートブック実行環境の前提となるソフトウェアのバージョン情報などは以下のとおりです。

* Notebook環境
  - [Jupyter Notebook for Literate Computing for Reproducible Infrastructure](https://github.com/NII-cloud-operation/Jupyter-LC_docker)
* VCP SDK
  - バージョン: 21.04.0
* Python
  - 3.7
* クラウドプロバイダ
  - AWS
  - Microsoft Azure

# VCP SDK の利用にあたって

VCP SDK を使ったチュートリアル用ノートブックを利用するために知っておいたほうがよい概念や仕組みについて説明します。

## オンデマンド構築サービスと VCP  (Virtual Cloud Provider) 

オンデマンド構築サービスでは、マルチクラウドを対象とした計算資源などを楽に管理するために VCP (Virtual Cloud Provider) というソフトウェアを開発し、使用しています。

この VCP といくつかの関連ソフトウェアを組み合わせた構成で、仮想クラウドコントローラ (VCコントローラ) を提供しています。

- 構築・管理ソフトウェア VCP 
  * クラウドプロバイダごとの API を抽象化し、制御・管理を容易にするソフトウェア

- オンデマンド構築機能
  * VCP と関連ソフトウェアを組み合わせた「 Virtual Cloud (VC) コントローラ」を提供

- VCコントローラの主な役割
  * クラウドAPI操作
  * クラウド(間)ネットワーク接続（仮想ルータ）
  * Dockerコンテナでのアプリケーション配備
  * アプリケーション実行環境のモニタリング

![](../vcp-expert/media/ocs-figure_01.png)

## VCP のサービスインターフェース

VCPは2種類のサービスインターフェースを提供しています。

- VCP REST API
  * 詳細な構築・管理が可能な REST インタフェース
  * YAML形式でクラウド構成情報を記述してPOST

- VCP SDK
  * Python3 ベースの開発キット
  * upyter Notebook 環境からの利用も可能

## VCP SDK

  ![VCP SDK](images/000-003.png)

* VCPを操作するための Python3 ライブラリ
* VCコントローラの REST API に対するラッパーライブラリ(vcplib)を呼び出している
* クラウドごとの制御APIの差異を吸収
* 仮想マシン(Amazon EC2インスタンス, Microsoft Azure VM, ...)などの計算資源と、Amazon EBS, Microsoft Azure Managed Disksなどの記録媒体を作成、削除することができる

<!---
VCP SDK の詳細については「[VCPSDK APIドキュメント](/handson/view/DOC/index.html)」から確認できます。
--->

## 基本的な構成要素

VCP SDK が扱う構成要素を以下に記します。

* VC (Virtual Cloud)
  - 複数のクラウドにまたがって構築されたひとつの仮想システム環境
* UnitGroup
  - 複数のUnitをまとめて扱うためのものです
  - サーバ、クライアントなどの相互に関係をもつ異質なNode群をグループとしてまとめます
  - `compute` と `storage` の2つのタイプがあります
    + `compute` は仮想マシンなどの計算資源で構成されています
    + `storage` はブロックデバイスなどの記録媒体で構成されています
* Unit
  - 同質（同じ計算資源(cpu, memory, ...)、同じクラウド、同じ用途、...）であるNodeをまとめて扱うためのものです
  - Unitに属するNodeをスケールアウト、スケールインすることができます
* Node
  - UnitGroupを構成する個々のノード
  - UnitGroupのタイプが `compute`の場合
    - [Amazon EC2インスタンス](https://aws.amazon.com/jp/ec2/), [Microsoft Azure VM](https://azure.microsoft.com/ja-jp/services/virtual-machines/) など
    - VCP SDKではこのNodeをVCノードと呼びます
  - UnitGroupのタイプが `storage`の場合
    - [Amazon EBS](https://aws.amazon.com/jp/ebs/), [Microsoft Azure Managed Disks](https://azure.microsoft.com/ja-jp/services/storage/disks/) など
    - VCP SDKではこのNodeをVCディスクと呼びます

VC, UnitGroup, Unit の例を以下の図に示します。

![VCP](images/000-001.png)

* UnitGroup1: ３つの Unit から構成されている `compute` タイプのUnitGroup
  - Unit1A
    * Amazon EC2 m5.large インスタンスで構成されている Unit
  - Unit1B
    * Amazon EC2 c5.large インスタンスで構成されている Unit
  - Unit1C
    * Microsoft Azure VM NV6 で構成されている Unit
* UnitGroup2: ３つの Unit から構成されている compute タイプのUnitGroup
  - Unit2A
    * Amazon EC2 m5.large で構成されている Unit
  - Unit2B
    * Amazon EC2 m5.large で構成されている Unit
  - Unit2C
    * Microsoft Azure VM D2 v3 で構成されている Unit
* UnitGroup3: １つの Unit から構成されている storage タイプのUnitGroup
  - Unit3
    * Amazon EBS から構成されている Unit

## VCノードについて

VCPの計算資源として利用できる環境はパブリッククラウドの仮想マシン(Amazon EC2, Microsoft Azure VM, ...)やベアメタル(BM)など多岐にわたっています。VCPではそのいずれに対しても同一の環境を提供できるようにするために[Docker コンテナ](https://www.docker.com/)を利用しています。

クラウド上のVM/BMをVCノードとして組み入れる際にVCPが各VM/BMに共通環境となるコンテナを起動します。VCPではこのコンテナのことを **Baseコンテナ** と呼んでいます。またBaseコンテナはVCノードのモニタリングなどのVCPの共通機能を提供するための役割も担っています。

![BC](images/000-002.png)

標準のBaseコンテナは以下のような特徴を有しています。

* [Alpine Linux コンテナ](https://hub.docker.com/_/alpine/)
* sshでログイン可能
* docker 実行環境(Docker in Docker)

標準のBaseコンテナは Alpine Linuxベースですが、他に Ubuntuベース、CentOSベースのものも用意しています。
また汎用的なBaseコンテナ以外にNFSサーバやGPU利用などの特定目的をもったBaseコンテナを用意しています。

# ハンズオン環境へのアクセス

ハンズオン環境のポータルサイトは、[JupyterHub](https://jupyter.org/hub) に講義演習のためのモジュール群を追加した形で NII のクラウド運用チームにより開発された [CoursewareHub](https://github.com/NII-cloud-operation/CoursewareHub-LC_platform) を使用して構築されています。

## システム構成

ハンズオンのNotebook環境はNIIのCoursewareHub上にあり、VCコントローラやVCノードにはAWSのパブリックサブネットに配置したNATインスタンスのリバースプロキシ(Nginx)を経由してアクセスできるように設定されています。

VCP SDKによって起動されるVCノードはプライベートサブネットに配置されます。

![ハンズオン環境](images/000-004-handson.png)

## VCコントローラ利用者の認証

VCP SDK を使ってVCコントローラにアクセスする際、アクセストークンで各利用者を認証します。

- オンデマンド構築サービスにおいては、VCコントローラの管理者用Web UIから利用者用のアクセストークンを払い出すことができます。
  - 管理者用Web UIの認証には、[学認 (GakuNin)](https://www.gakunin.jp/) を利用

----
- **ハンズオン環境では、事前に発行済のトークンを使用します。** ⇒ハンズオン用トークンファイルはこちら [token.txt](../../../files/token.txt)

# 準備

この後に実行するNotebookで必要となる準備作業を行います。

## SSH公開鍵認証のための鍵ファイル作成

作成したSSHの鍵ファイルはこのNotebook環境からVCノードにログインする際に利用します。

次のセルを実行してください。

In [ ]:
!mkdir -p -m 700 ~/.ssh
!touch ~/.ssh/known_hosts && chmod 600 ~/.ssh/known_hosts
!test -f ~/.ssh/id_rsa || ssh-keygen -t rsa -f ~/.ssh/id_rsa -N ""
!ls -la ~/.ssh

## 利用するクラウドのクレデンシャル設定

次に、クラウドプロバイダのクレデンシャル情報を秘密情報管理サーバ (Vaultサーバ) に書き込みます。設定したクレデンシャル情報は、これから実行するNotebookでのクラウドプロバイダ操作に使用されます。

JupyterNotebook の Terminal 上で、以下のコマンドを **コピー＆ペースト** して実行します。

```
sh vault.sh $(cat token.txt)
```

- `vault.sh` スクリプトは、VCP の認証に必要なトークンに関連付けてクレデンシャル情報を Vault サーバに保存します。
- ハンズオン用トークンファイルはこちらにあります→ [token.txt](../../../files/token.txt)

以下のセルを実行すると、JupyterNotebook の Terminal を開くためにリンクが作成されます。

In [ ]:
user = !whoami

from IPython.display import HTML
HTML('<a target="_blank" href="/user/{}/terminals/1">Open Terminal</a>'.format(user[0]))

# Notebook一覧

次のセルを実行するとNotebookの一覧が表示されます。

<!---
生成するには
```
from IPython.display import SVG
%run scripts/generate-diagram.py
SVG(filename=generate_svg_diag(diag='images/notebooks.diag', nb_dir='.', verbose=False))
```
--->

In [ ]:
from IPython.core.display import SVG
SVG(filename='images/notebooks.svg')